In [1]:
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.solvers.mip_solver import mip_solve
from cloudmanufacturing.validation import construct_delta, objvalue

from joblib import Parallel, delayed
from tqdm import trange
import pandas as pd
import numpy as np
import random
import openpyxl
from cloudmanufacturing.data_generation import create_excel_table
random.seed(42)

c:\Users\dmitrii\miniconda3\envs\graph\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Data generation

In [3]:
N = 100
file_path = '../data/train_data.xlsx'

trios = []
for i in range(N):
    trios.append(tuple((random.randint(5,15),random.randint(5, 20),random.randint(5,15))))
trios.sort()

wb = openpyxl.Workbook()
del wb['Sheet']

for n_operations, n_suboperations, n_cities in trios:
    wb = create_excel_table(wb, n_operations, n_suboperations, n_cities, n_problem=1)

wb.save(file_path)

#### Fatahi

In [11]:
dataset = read_fatahi_dataset("../data/fatahi.xlsx")

100%|██████████| 18/18 [00:00<00:00, 247.84it/s]


In [12]:
def fatahi_problem(i, problem):
    print(problem['n_operations'])
    print(problem['n_suboperations'])
    print(problem['n_cities'])
    delta, gamma, status, value = mip_solve(problem)

    if status.name == 'OPTIMAL':
        np.save(f'./fatahi_solutions/delta_{i}_op.npy', delta)
        np.save(f'./fatahi_solutions/gamma_{i}_op.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    if status.name == 'FEASIBLE':
        np.save(f'./fatahi_solutions/delta_{i}_fb.npy', delta)
        np.save(f'./fatahi_solutions/gamma_{i}_fb.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    print(i," wasn't solved")
    return None

In [13]:
sum_info = Parallel(n_jobs=-1)(
    delayed(fatahi_problem)(i, dataset[i]) for i in trange(len(dataset))
)
sum_info = [info for info in sum_info if info is not None]

100%|██████████| 18/18 [00:00<?, ?it/s]


### Train dataset

In [14]:
dataset_train = read_fatahi_dataset("../data/train_data.xlsx")

100%|██████████| 100/100 [00:08<00:00, 11.24it/s]


In [15]:
def train_problem(i, problem):
    delta, gamma, status, value = mip_solve(problem)

    if status.name == 'OPTIMAL':
        np.save(f'./train_solutions/delta_{i}_op.npy', delta)
        np.save(f'./train_solutions/gamma_{i}_op.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    if status.name == 'FEASIBLE':
        np.save(f'./train_solutions/delta_{i}_fb.npy', delta)
        np.save(f'./train_solutions/gamma_{i}_fb.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    print(i," wasn't solved")
    return None

In [16]:
sum_info_train = Parallel(n_jobs=-1)(
    delayed(train_problem)(i, dataset_train[i]) for i in trange(len(dataset_train))
)
sum_info_train = [info for info in sum_info_train if info is not None]

100%|██████████| 100/100 [05:12<00:00,  3.13s/it]
